In [ ]:
# imports
from importlib import reload

import os # a module to interact with the operating system
os.environ["ENDPOINT_URL"]="http://rook-ceph-rgw-nautiluss3.rook"
import numpy as np

import pandas as pd
import xarray
import h5py
import healpy as hp
import time

import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from matplotlib import pyplot as plt
import seaborn as sns
import scipy.stats as stats

from ulmo import io as ulmo_io
from ulmo import plotting
from ulmo.llc import io as llc_io
from ulmo.llc import plotting as llc_plotting

from ulmo.utils import image_utils
from ulmo.analysis import figures as ulmo_figs
from ulmo.plotting import plotting as ulmo_plotting
from ulmo.analysis import spatial_plots as sp

#### Goals: 
#### 1. Select LLC and VIIRS cutouts from two regions: in equatorial pacific and 60W shelf/confluence
#### 2. Create a side by side 9 ~ med LL image gallery for LLC and VIIRS

# Load Tables

In [ ]:
llc_tbl= ulmo_io.load_main_table( 's3://llc/Tables/LLC_uniform_viirs_noise.parquet' )
llc_tbl.head()

In [ ]:
uni_tbl = ulmo_io.load_main_table( 's3://llc/Tables/test_uniform_r0.5_test.feather' )
uni_tbl.head()

In [ ]:
viirs_tbl = ulmo_io.load_main_table( 's3://viirs/Tables/VIIRS_all_99clear_std.parquet')
viirs_tbl.head()

# Region 1 : Equatorial Pacific

### North : LLC

In [ ]:
eqtr_n = (llc_tbl.lat > 0. ) & (np.abs(llc_tbl.lat) < 2.) & (np.abs(llc_tbl.lon + 100) < 5.)
llc_eqtr_n = llc_tbl[ eqtr_n ]

med_LL_n = np.median(llc_eqtr_n.LL.to_numpy())
med_LL_n

In [ ]:
llc_eqtr_n.keys()

### South : LLC

In [ ]:
eqtr_s = (llc_tbl.lat < 0. ) & (np.abs(llc_tbl.lat) < 2.) & (np.abs(llc_tbl.lon + 100) < 5.)
llc_eqtr_s = llc_tbl[ eqtr_s ]

med_LL_s = np.median(llc_eqtr_s.LL.to_numpy())
med_LL_s

In [ ]:
med = np.abs(llc_tbl.LL - med_LL_n) < 50
llc_n = llc_tbl[ eqtr_n & med ]
llc_n

In [ ]:
med = np.abs(llc_tbl.LL - med_LL_s) < 50
llc_s = llc_tbl[ eqtr_s & med ]
llc_s

### North : VIIRS

In [ ]:
eqtr_n = (viirs_tbl.lat > 0. ) & (np.abs(viirs_tbl.lat) < 2.) & (np.abs(viirs_tbl.lon + 100) < 5.)
viirs_eqtr_n = viirs_tbl[ eqtr_n ]

med_LL_n = np.median(viirs_eqtr_n.LL.to_numpy())
med_LL_n

### South : VIIRS

In [ ]:
eqtr_s = (viirs_tbl.lat < 0. ) & (np.abs(viirs_tbl.lat) < 2.) & (np.abs(viirs_tbl.lon + 100) < 5.)
viirs_eqtr_s = viirs_tbl[ eqtr_s ]

med_LL_s = np.median(viirs_eqtr_s.LL.to_numpy())
med_LL_s

In [ ]:
med = np.abs(viirs_tbl.LL - med_LL_n) < 50
viirs_n = viirs_tbl[ eqtr_n & med ]
viirs_n

In [ ]:
med = np.abs(viirs_tbl.LL - med_LL_s) < 50
viirs_s = viirs_tbl[ eqtr_s & med ]
viirs_s

## Check Seasonal Bias of Viirs

In [ ]:
def fig_year_month(outfile, ptype, evals_tbl=None, frac=False,
                   all=False):
    """
    Time evolution in outliers
    Parameters
    ----------
    outfile
    ptype
    evals_tbl
    Returns
    -------
    """

    # Load
    if evals_tbl is None:
        evals_tbl = results.load_log_prob(ptype, feather=True)
        print("Loaded..")

    # Outliers
    point1 = int(0.001 * len(evals_tbl))
    isortLL = np.argsort(evals_tbl.LL)
    outliers = evals_tbl.iloc[isortLL[0:point1]]

    # All
    if all or frac:
        all_years = [item.year for item in evals_tbl.datetime]
        all_months = [item.month for item in evals_tbl.datetime]

    # Parse
    years = [item.year for item in outliers.datetime]
    months = [item.month for item in outliers.datetime]

    # Histogram
    bins_year = np.arange(2011.5, 2021.5)
    bins_month = np.arange(0.5, 13.5)

    counts, xedges, yedges = np.histogram2d(months, years,
                                            bins=(bins_month, bins_year))
    if all or frac:
        all_counts, _, _ = np.histogram2d(all_months, all_years,
                                            bins=(bins_month, bins_year))

    fig = plt.figure(figsize=(12, 8))
    plt.clf()
    gs = plt.GridSpec(5,6)

    # Total NSpax
    ax_tot = plt.subplot(gs[1:,1:-1])

    cm = plt.get_cmap('Blues')
    if frac:
        values  = counts.transpose()/all_counts.transpose()
        lbl = 'Fraction'
    elif all:
        cm = plt.get_cmap('Greens')
        norm = np.sum(all_counts) / np.product(all_counts.shape)
        values = all_counts.transpose()#/norm
        lbl = 'Counts'
        #lbl = 'Fraction (all)'
    else:
        values = counts.transpose()
        lbl = 'Counts'
    mplt = ax_tot.pcolormesh(xedges, yedges, values, cmap=cm)
    
    # Color bar
    cbaxes = fig.add_axes([0.03, 0.1, 0.05, 0.7])
    cb = plt.colorbar(mplt, cax=cbaxes, aspect=20)
    #cb.set_label(lbl, fontsize=20.)
    cbaxes.yaxis.set_ticks_position('left')
    cbaxes.set_xlabel(lbl, fontsize=15.)

    ax_tot.set_xlabel('Month')
    ax_tot.set_ylabel('Year')

    #ax_tot.set_fontsize(ax_tot, 19.)

    # Edges
    fsz = 30.
    months = np.mean(values, axis=0)
    ax_m = plt.subplot(gs[0,1:-1])
    ax_m.step(np.arange(12)+1, months, color='k', where='mid')
    #ax_m.set_fontsize(ax_m, fsz)
    #ax_m.minorticks_on()

    years = np.mean(values, axis=1)
    ax_y = plt.subplot(gs[1:,-1])
    ax_y.invert_xaxis()
    ax_y.step(years, 2012 + np.arange(9), color='k', where='mid')
    #ax_y.set_xlim(40,80)
    #ax_y.set_fontsize(ax_y, fsz)
    
    # Layout and save
    plt.title('Total # of Cutouts : {}'.format( round(np.sum(all_counts))))
    plt.tight_layout(pad=0.2,h_pad=0.2,w_pad=0.1)
    plt.show()
    #plt.savefig('mbyytest', dpi=600)
    #plt.close()
    #print('Wrote {:s}'.format(outfile))


In [ ]:
fig_year_month(outfile = viirs_n, ptype='std', evals_tbl= viirs_n, all=True)

In [ ]:
fig_year_month(outfile = viirs_eqtr_n, ptype='std', evals_tbl= viirs_eqtr_n, all=True)

In [ ]:
fig_year_month(outfile = viirs_s, ptype='std', evals_tbl= viirs_s, all=True)

In [ ]:
fig_year_month(outfile = viirs_eqtr_s, ptype='std', evals_tbl= viirs_eqtr_s, all=True)

# Region 2: 60W Shelf & Confluence

In [ ]:
R2 = dict( lon=-61.0, dlon=1., lat=-45, dlat=2)
R1 = dict( lon=-56.5, dlon=1.5, lat=-45, dlat=2)

### R1 & R2 : LLC

In [ ]:
in_R1, in_R2 = [((np.abs(llc_tbl.lon.values - R['lon']) < R['dlon']) & (np.abs(llc_tbl.lat.values - R['lat']) < R['dlat'])) for R in [R1, R2]]


#### R1

In [ ]:
llc_R1 = llc_tbl[ in_R1 ]
med_LL1 = np.median(llc_R1.LL.to_numpy())
med_LL1

In [ ]:
med = np.abs( llc_tbl.LL - med_LL1) < 20
llc1 = llc_tbl[ in_R1 & med ]
llc1

#### R2

In [ ]:
llc_R2 = llc_tbl[ in_R2 ]
med_LL2 = np.median(llc_R2.LL.to_numpy())
med_LL2

In [ ]:
med = np.abs( llc_tbl.LL - med_LL2) < 20
llc2 = llc_tbl[ in_R2 & med ]
llc2

### R1 & R2 : VIIRS

In [ ]:
in_R1, in_R2 = [((np.abs(viirs_tbl.lon.values - R['lon']) < R['dlon']) & (np.abs(viirs_tbl.lat.values - R['lat']) < R['dlat'])) for R in [R1, R2]]

#### R1

In [ ]:
viirs_R1 = viirs_tbl[ in_R1 ]
med_V1 = np.median( viirs_R1.LL.to_numpy())
med_V1

In [ ]:
med = np.abs( viirs_tbl.LL - med_V1) < 10
viirs1 = viirs_tbl[ in_R1 & med]
viirs1

#### R2

In [ ]:
viirs_R2 = viirs_tbl[ in_R2 ]
med_V2 = np.median( viirs_R2.LL.to_numpy())
med_V2

In [ ]:
med = np.abs( viirs_tbl.LL - med_V2) < 20
viirs2 = viirs_tbl[ in_R2 & med]
viirs2

In [ ]:
_ = ulmo_figs.show_spatial(viirs2, lbl = 'viirs_2')

In [ ]:
_ = ulmo_figs.show_spatial(viirs1, lbl = 'viirs_R1')

# ~ Med LL 9 image gallery

In [ ]:
pal, cm = plotting.load_palette()

### Equatorial Pacific: LLC

In [ ]:
#Generate 9 random numbers from tbl
northlist1 = np.random.choice( llc_n.index.to_numpy(), size = 9)
print(northlist1)

In [ ]:
northlist1 = [312928, 703391, 312559, 573114,  51624, 311433, 181160, 573482, 182656]

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (10,8) )

fig.suptitle('9 ~ med LL imgs: North Pacific: LLC data')
fig.supylabel('dT')

for i in [0, 1, 2]: 
    
    idx = northlist1[ i ] 
    cutout = llc_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax=axes[i, 0], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)
    
for i in [3, 4, 5]: 
    
    idx = northlist1[ i ] 
    cutout = llc_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax = axes[i-3, 1], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)
    
for i in [6, 7, 8]: 
    
    idx = northlist1[ i ] 
    cutout = llc_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax = axes[i-6, 2], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)  
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (8,8) )

fig.suptitle('9 ~ med LL imgs: North Pacific: LLC data')
fig.supylabel('dT')
cbar_ax = fig.add_axes([0.95, 0.15, 0.02, 0.7])
    
for i, ax in enumerate(axes.flat):
    idx = northlist1[ i ] 
    cutout = llc_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax=ax, data=img, xticklabels=[], yticklabels=[], cmap=cm,
                cbar=i == 0,
                vmin=-1., vmax=2.,
                cbar_ax=None if i else cbar_ax)
    ax.set_title('LL = {}'.format(round(cutout.LL)))

fig.tight_layout(rect=[0, 0, .9, 1])

plt.savefig('med_LL_imgs_LC_NP', dpi = 600)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (8,8) )

fig.suptitle('9 ~ med LL imgs: North Pacific: LLC data')
fig.supylabel('dT')
cbar_ax = fig.add_axes([0.95, 0.15, 0.02, 0.7])
    
for i, ax in enumerate(axes.flat):
    idx = northlist[ i ] 
    cutout = uni_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax=ax, data=img, xticklabels=[], yticklabels=[], cmap=cm,
                cbar=i == 0,
                vmin=-1.5, vmax=2.2,
                cbar_ax=None if i else cbar_ax)
    ax.set_title('LL = {}'.format(round(cutout.LL)))

fig.tight_layout(rect=[0, 0, .9, 1])

In [ ]:
#Generate 9 random numbers from tbl
southlist1 = np.random.choice( llc_s.index.to_numpy(), size = 9)
print(southlist1)

In [ ]:
southlist1 = [184128, 184122, 182657, 314404, 182648,  53492, 183760, 574962, 704128]

In [ ]:
viirs_tbl.iloc[0]

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (10,8) )

fig.suptitle('9 ~ Med LL Imgs: South Pacific: LLC data')
fig.supylabel( 'dT' )

for i in [0, 1, 2]: 
    
    idx = southlist1[ i ] 
    cutout = llc_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax=axes[i, 0], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)
    
for i in [3, 4, 5]: 
    
    idx = southlist1[ i ] 
    cutout = llc_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax = axes[i-3, 1], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)
    
for i in [6, 7, 8]: 
    
    idx = southlist1[ i ] 
    cutout = llc_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax = axes[i-6, 2], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)  
plt.show()
#plt.savefig( 'medLL9_LLC_SP', dpi = 600)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (8,8) )

fig.suptitle('9 ~ med LL imgs: South Pacific: LLC data')
fig.supylabel('dT')
cbar_ax = fig.add_axes([0.95, 0.15, 0.02, 0.7])
    
for i, ax in enumerate(axes.flat):
    idx = southlist1[ i ] 
    cutout = llc_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax=ax, data=img, xticklabels=[], yticklabels=[], cmap=cm,
                cbar=i == 0,
                vmin=-1.5, vmax=1.5,
                cbar_ax=None if i else cbar_ax)
    ax.set_title('LL = {}'.format(round(cutout.LL)))

fig.tight_layout(rect=[0, 0, .9, 1])
plt.savefig( 'med_LL_LLC_SP', dpi = 600)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (8,8) )

fig.suptitle('9 ~ med LL imgs: South Pacific: LLC data')
fig.supylabel('dT')
cbar_ax = fig.add_axes([0.95, 0.15, 0.02, 0.7])
    
for i, ax in enumerate(axes.flat):
    idx = southlist[ i ] 
    cutout = uni_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax=ax, data=img, xticklabels=[], yticklabels=[], cmap=cm,
                cbar=i == 0,
                vmin=-1.3, vmax=1.3,
                cbar_ax=None if i else cbar_ax)
    ax.set_title('LL = {}'.format(round(cutout.LL)))

fig.tight_layout(rect=[0, 0, .9, 1])

### Equatorial Pacific: VIIRS

In [ ]:
#Generate 9 random numbers from tbl
northlist = np.random.choice( viirs_n.index.to_numpy(), size = 9)
print(northlist)

In [ ]:
northlist=[1553120, 1908931, 1309681, 1054985, 2212253, 2196295, 1156313, 1007509, 1054985]

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (10,8) )

fig.suptitle('9 ~ Med LL imgs: North Pacific: VIIRS data')
fig.supylabel('dT')

for i in [0, 1, 2]: 
    
    idx = northlist[ i ] 
    cutout = viirs_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax=axes[i, 0], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)
    
for i in [3, 4, 5]: 
    
    idx = northlist[ i ] 
    cutout = viirs_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax = axes[i-3, 1], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)
    
for i in [6, 7, 8]: 
    
    idx = northlist[ i ] 
    cutout = viirs_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax = axes[i-6, 2], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)  
plt.show()
#plt.savefig( 'medLL9_VIIRS_NP', dpi = 600)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (8,8) )

fig.suptitle('9 ~ med LL imgs: North Pacific: VIIRS data')
fig.supylabel('dT')
cbar_ax = fig.add_axes([0.95, 0.15, 0.02, 0.7])
    
for i, ax in enumerate(axes.flat):
    idx = northlist[ i ] 
    cutout = viirs_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    name1 = cutout.pp_file.replace('s3://viirs/PreProc/VIIRS_', '')
    year = name1[0:4]
    
    sns.heatmap(ax=ax, data=img, xticklabels=[], yticklabels=[], cmap=cm,
                cbar=i == 0,
                vmin=-1.0, vmax=2.0,
                cbar_ax=None if i else cbar_ax)
    #ax.set_title('i={}, yr={}'.format(cutout.pp_idx, year))
    ax.set_title('LL = {}'.format(round(cutout.LL)))

fig.tight_layout(rect=[0, 0, .9, 1])
plt.savefig( 'med_LL_VIIRS_NP', dpi=600)

In [ ]:
viirs_tbl.iloc

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.clf()

tformM = ccrs.Mollweide()
tformP = ccrs.PlateCarree()

ax = plt.axes(projection=tformM)
    
ax.coastlines(zorder=10)
ax.set_extent([-105, -95, -2, 2], ccrs.PlateCarree())
    
gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=2, x_inline=False, y_inline=False, 
        color='black', alpha=0.5, linestyle=':', draw_labels=True)
gl.top_labels=False
gl.bottom_labels=True
gl.left_labels=True
gl.right_labels=False
#gl.xformatter = LONGITUDE_FORMATTER
#gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'color': 'black','weight': 'bold'}
gl.ylabel_style = {'color': 'black', 'weight': 'bold'}

for i, elem in enumerate(northlist): 
    
    cutout = viirs_tbl.iloc[elem]
    lat = cutout.lat
    lon = cutout.lon
    
    plt.plot(lon, lat, marker = 'X', color= 'orange', markersize = 10, transform=ccrs.PlateCarree())
    #plt.text(lon+0.2, lat, '{}'.format(i), horizontalalignment='left',
         #transform=ccrs.Geodetic())

for i, elem in enumerate(northlist1): 
    
    cutout = llc_tbl.iloc[elem]
    lat = cutout.lat
    lon = cutout.lon
    
    plt.plot(lon, lat, marker = 'P', color= 'seagreen', markersize = 10, transform=ccrs.PlateCarree())
    #plt.text(lon+0.2, lat, '{}'.format(i), horizontalalignment='left',
         #transform=ccrs.Geodetic())
    

In [ ]:
#Generate 9 random numbers from tbl
southlist = np.random.choice( viirs_s.index.to_numpy(), size = 9)
print(southlist)

In [ ]:
southlist = [ 441085, 134431, 1814813, 2141796, 1701056, 1952871, 1391020, 1147233, 1222688]

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (10,8) )

fig.suptitle('9 ~ med LL imgs: South Pacific: VIIRS data')
fig.supylabel('dT')

for i in [0, 1, 2]: 
    
    idx = southlist[ i ] 
    cutout = viirs_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax=axes[i, 0], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)
    
for i in [3, 4, 5]: 
    
    idx = southlist[ i ] 
    cutout = viirs_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax = axes[i-3, 1], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)
    
for i in [6, 7, 8]: 
    
    idx = southlist[ i ] 
    cutout = viirs_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax = axes[i-6, 2], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)  
plt.show()
#plt.savefig( 'medLL9_VIIRS_SP', dpi = 600)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (8,8) )

fig.suptitle('9 ~ med LL imgs: South Pacific: VIIRS data')
fig.supylabel('dT')
cbar_ax = fig.add_axes([0.95, 0.15, 0.02, 0.7])
    
for i, ax in enumerate(axes.flat):
    idx = southlist[ i ] 
    cutout = viirs_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    name1 = cutout.pp_file.replace('s3://viirs/PreProc/VIIRS_', '')
    year = name1[0:4]
    
    sns.heatmap(ax=ax, data=img, xticklabels=[], yticklabels=[], cmap=cm,
                cbar=i == 0,
                vmin=-1.5, vmax=1.5,
                cbar_ax=None if i else cbar_ax)
    #ax.set_title('i={}, yr={}'.format(cutout.pp_idx, year))
    ax.set_title('LL = {}'.format(round(cutout.LL)))

fig.tight_layout(rect=[0, 0, .9, 1])
plt.savefig('med_LL_VIIRS_SP', dpi=600)

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.clf()

tformM = ccrs.Mollweide()
tformP = ccrs.PlateCarree()

ax = plt.axes(projection=tformM)
    
ax.coastlines(zorder=10)
ax.set_extent([-105, -95, -2, 2], ccrs.PlateCarree())
    
gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=2, x_inline=False, y_inline=False, 
        color='black', alpha=0.5, linestyle=':', draw_labels=True)
gl.top_labels=False
gl.bottom_labels=True
gl.left_labels=True
gl.right_labels=False
#gl.xformatter = LONGITUDE_FORMATTER
#gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'color': 'black', 'size': 14, 'weight': 'bold'}
gl.ylabel_style = {'color': 'black', 'size': 14, 'weight': 'bold'}


for i, elem in enumerate(northlist): 
    
    cutout = viirs_tbl.iloc[elem]
    lat = cutout.lat
    lon = cutout.lon
    
    if i == 8: 
    
        plt.plot(lon, lat, marker = 'o', color= 'orange', markersize = 10, transform=ccrs.PlateCarree(), label = 'VIIRS')
    #plt.text(lon+0.2, lat, '{}'.format(i), horizontalalignment='left',
         #transform=ccrs.Geodetic())
        
        plt.legend()
    else: 
        
        plt.plot(lon, lat, marker = 'o', color= 'orange', markersize = 10, transform=ccrs.PlateCarree())
    #plt.text(lon+0.2, lat, '{}'.format(i), horizontalalignment='left',
         #transform=ccrs.Geodetic())
    
for i, elem in enumerate(northlist1): 
    
    cutout = llc_tbl.iloc[elem]
    lat = cutout.lat
    lon = cutout.lon
    
    if i == 8: 
    
        plt.plot(lon, lat, marker = '*', color= 'seagreen', markersize = 10, transform=ccrs.PlateCarree(), label = 'LLC')
    #plt.text(lon+0.2, lat, '{}'.format(i), horizontalalignment='left',
         #transform=ccrs.Geodetic())
        
        plt.legend(prop={'size': 14, 'weight': 'bold'})
    else: 
        
        plt.plot(lon, lat, marker = '*', color= 'seagreen', markersize = 10, transform=ccrs.PlateCarree())
    #plt.text(lon+0.2, lat, '{}'.format(i), horizontalalignment='left',
         #transform=ccrs.Geodetic())

for i, elem in enumerate(southlist): 
    
    cutout = viirs_tbl.iloc[elem]
    lat = cutout.lat
    lon = cutout.lon
    
    plt.plot(lon, lat, marker = 'o', color= 'orange', markersize = 10, transform=ccrs.PlateCarree())
    #plt.text(lon+0.2, lat, '{}'.format(i), horizontalalignment='left',
         #transform=ccrs.Geodetic())
    
for i, elem in enumerate(southlist1): 
    
    cutout = llc_tbl.iloc[elem]
    lat = cutout.lat
    lon = cutout.lon
    
        
    plt.plot(lon, lat, marker = '*', color= 'seagreen', markersize = 10, transform=ccrs.PlateCarree())
    #plt.text(lon+0.2, lat, '{}'.format(i), horizontalalignment='left',
         #transform=ccrs.Geodetic())
        
#plt.show()
plt.savefig('loc', dpi=600)

### 60W : LLC

In [ ]:
R1list = np.random.choice( llc1.index.to_numpy(), size = 9)
print(R1list)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (10,8) )

fig.suptitle('9 ~ Med LL imgs: 60W R1: LLC data')
fig.supylabel('dT')

for i in [0, 1, 2]: 
    
    idx = R1list[ i ] 
    cutout = llc_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax=axes[i, 0], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)
    
for i in [3, 4, 5]: 
    
    idx = R1list[ i ] 
    cutout = llc_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax = axes[i-3, 1], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)
    
for i in [6, 7, 8]: 
    
    idx = R1list[ i ] 
    cutout = llc_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax = axes[i-6, 2], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)  
plt.show()
#plt.savefig( 'medLL9_LLC_60W_R1', dpi = 600)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (8,8) )

fig.suptitle('9 ~ med LL imgs: 60W R1: LLC data')
fig.supylabel('dT')
cbar_ax = fig.add_axes([0.95, 0.15, 0.02, 0.7])
    
for i, ax in enumerate(axes.flat):
    idx = R1list[ i ] 
    cutout = llc_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    name1 = cutout.pp_file.replace('s3://viirs/PreProc/VIIRS_', '')
    year = name1[0:4]
    
    sns.heatmap(ax=ax, data=img, xticklabels=[], yticklabels=[], cmap=cm,
                cbar=i == 0,
                vmin=-4, vmax=5,
                cbar_ax=None if i else cbar_ax)
    #ax.set_title('i={}, yr={}'.format(cutout.pp_idx, year))
    ax.set_title('LL = {}'.format(round(cutout.LL)))

fig.tight_layout(rect=[0, 0, .9, 1])

In [ ]:
R2list = np.random.choice( llc2.index.to_numpy(), size = 9)
print(R2list)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (10,8) )

fig.suptitle('9 ~ Med LL imgs: 60W R2: LLC data')
fig.supylabel('dT')

for i in [0, 1, 2]: 
    
    idx = R2list[ i ] 
    cutout = llc_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax=axes[i, 0], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)
    
for i in [3, 4, 5]: 
    
    idx = R2list[ i ] 
    cutout = llc_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax = axes[i-3, 1], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)
    
for i in [6, 7, 8]: 
    
    idx = R2list[ i ] 
    cutout = llc_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax = axes[i-6, 2], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)  
plt.show()
#plt.savefig( 'medLL9_LLC_60W_R2', dpi = 600)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (8,8) )

fig.suptitle('9 ~ med LL imgs: 60W R2: LLC data')
fig.supylabel('dT')
cbar_ax = fig.add_axes([0.95, 0.15, 0.02, 0.7])
    
for i, ax in enumerate(axes.flat):
    idx = R2list[ i ] 
    cutout = llc_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    name1 = cutout.pp_file.replace('s3://viirs/PreProc/VIIRS_', '')
    year = name1[0:4]
    
    sns.heatmap(ax=ax, data=img, xticklabels=[], yticklabels=[], cmap=cm,
                cbar=i == 0,
                vmin=-3, vmax=2,
                cbar_ax=None if i else cbar_ax)
    #ax.set_title('i={}, yr={}'.format(cutout.pp_idx, year))
    ax.set_title('LL = {}'.format(round(cutout.LL)))

fig.tight_layout(rect=[0, 0, .9, 1])

### 60W : VIIRS

In [ ]:
R1list = np.random.choice( viirs1.index.to_numpy(), size = 9)
print(R1list)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (10,8) )

fig.suptitle('9 ~ Med LL imgs: 60W R1: VIIRS data')
fig.supylabel('dT')

for i in [0, 1, 2]: 
    
    idx = R1list[ i ] 
    cutout = viirs_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax=axes[i, 0], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)
    
for i in [3, 4, 5]: 
    
    idx = R1list[ i ] 
    cutout = viirs_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax = axes[i-3, 1], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)
    
for i in [6, 7, 8]: 
    
    idx = R1list[ i ] 
    cutout = viirs_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax = axes[i-6, 2], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)  
plt.show()
#plt.savefig( 'medLL9_VIIRS_60W_R1', dpi = 600)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (8,8) )

fig.suptitle('9 ~ med LL imgs: 60W R1: VIIRS data')
fig.supylabel('dT')
cbar_ax = fig.add_axes([0.95, 0.15, 0.02, 0.7])
    
for i, ax in enumerate(axes.flat):
    idx = R1list[ i ] 
    cutout = viirs_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    name1 = cutout.pp_file.replace('s3://viirs/PreProc/VIIRS_', '')
    year = name1[0:4]
    
    sns.heatmap(ax=ax, data=img, xticklabels=[], yticklabels=[], cmap=cm,
                cbar=i == 0,
                vmin=-4, vmax=5,
                cbar_ax=None if i else cbar_ax)
    #ax.set_title('i={}, yr={}'.format(cutout.pp_idx, year))
    ax.set_title('LL = {}'.format(round(cutout.LL)))

fig.tight_layout(rect=[0, 0, .9, 1])

In [ ]:
R2list = np.random.choice( viirs2.index.to_numpy(), size = 9)
print(R2list)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (10,8) )

fig.suptitle('9 ~ Med LL imgs: 60W R2: VIIRS data')
fig.supylabel('dT')

for i in [0, 1, 2]: 
    
    idx = R2list[ i ] 
    cutout = viirs_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax=axes[i, 0], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)
    
for i in [3, 4, 5]: 
    
    idx = R2list[ i ] 
    cutout = viirs_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax = axes[i-3, 1], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)
    
for i in [6, 7, 8]: 
    
    idx = R2list[ i ] 
    cutout = viirs_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    
    sns.heatmap(ax = axes[i-6, 2], data=img, xticklabels=[], yticklabels=[], cmap=cm,
             cbar=True)  
plt.show()
#plt.savefig( 'medLL9_LLC_60W_R2', dpi = 600)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (8,8) )

fig.suptitle('9 ~ med LL imgs: 60W R2: VIIRS data')
fig.supylabel('dT')
cbar_ax = fig.add_axes([0.95, 0.15, 0.02, 0.7])
    
for i, ax in enumerate(axes.flat):
    idx = R2list[ i ] 
    cutout = viirs_tbl.iloc[ idx ] 
    
    img= llc_io.grab_image(cutout)
    name1 = cutout.pp_file.replace('s3://viirs/PreProc/VIIRS_', '')
    year = name1[0:4]
    
    sns.heatmap(ax=ax, data=img, xticklabels=[], yticklabels=[], cmap=cm,
                cbar=i == 0,
                vmin=-3, vmax=2,
                cbar_ax=None if i else cbar_ax)
    #ax.set_title('i={}, yr={}'.format(cutout.pp_idx, year))
    ax.set_title('LL = {}'.format(round(cutout.LL)))

fig.tight_layout(rect=[0, 0, .9, 1])